In [ ]:
!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

--2021-05-24 21:37:16--  https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 410039235 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz.2’

pytorch_weights.tar 100%[===================>] 391.04M   794KB/s    in 2m 56s  

2021-05-24 21:40:13 (2.22 MB/s) - ‘pytorch_weights.tar.gz.2’ saved [410039235/410039235]

--2021-05-24 21:40:13--  https://users.dcc.uchile.cl/~jperez/beto/uncased_2M/vocab.txt
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248047 (242K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 242.23K   485KB/s  

In [2]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
# Evaluation
import pytorch
import torch
from torch import nn
# import tensorflow_text as text
import pandas as pd
# from official.nlp import optimization  # to create AdamW optimizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import np_utils
from keras import backend as K
# from focal_loss import BinaryFocalLoss
import matplotlib.pyplot as plt
from tensorflow.keras import activations
# import tensorflow_addons as tfa


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

ModuleNotFoundError: ignored

In [7]:
# Read data
url = 'https://www.fing.edu.uy/inco/grupos/pln/haha/2019/data/haha_2019_train.csv'
url2 = 'https://www.fing.edu.uy/inco/grupos/pln/haha/2019/data/haha_2019_test_gold.csv'
train = pd.read_csv(url)
testG = pd.read_csv(url2)


train = train.loc[train['is_humor'] == 1].copy()
testG = testG.loc[testG['is_humor'] == 1].copy()

In [6]:
testG

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,funniness_average
0,1092844370540331008,Historia #36.\n*Entra corriendo y gritando al ...,0,3,1,0,0,0,0,NaN
1,1092879430748459009,estoy tomando helado en el centro y pedi tiram...,0,3,0,0,0,0,0,NaN
2,1092537811419516933,— ¿De 15 o 20 centímetros?\n\n— Ven y averígua...,1,0,3,1,1,0,0,1.6
3,1092866638112792581,Que fea soy,0,3,0,0,0,0,0,NaN
4,332525498250969088,"#OMG, VAMOS EN RETROCESO RESULTE CON 40 SEGUID...",0,4,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
5995,1092852595570425856,#ElPulso señor @oscarrenteriaj La corbata ya n...,0,3,0,0,0,0,0,NaN
5996,641393345181425664,¿Hace frío? Acá en el infierno que transformas...,0,3,0,0,0,0,0,NaN
5997,988378375118053376,Buenas días mis cariños y cariñas,0,3,0,0,0,0,0,NaN
5998,777986884895019009,"""Les digo ven, ven, ven, animalito ven, ven y ...",0,3,2,0,0,0,0,NaN


In [8]:
BATCH_SIZE = 32
MAX_SEQUENCE_LENGTH = 125
bert_tokenizer = BertTokenizer.from_pretrained("pytorch/",do_lower_case=True)
bert_model = BertForSequenceClassification.from_pretrained("pytorch/",num_labels=2).to("cuda")

# BATCH_SIZE = 32
# MAX_SEQUENCE_LENGTH = 125
# bert_tokenizer = BertTokenizer.from_pretrained("pytorch/",do_lower_case=True)
# bert_model = BertForSequenceClassification.from_pretrained("pytorch/",num_labels=1).to("cuda")

NameError: ignored

In [ ]:
x_train, x_dev,y_train, y_dev = train_test_split(train["text"].tolist(),train["funniness_average"].tolist(), test_size=0.2,stratify=train["is_humor"])

x_test = testG["text"].tolist()
y_test = testG["funniness_average"].tolist()



train_encod = bert_tokenizer(x_train,truncation=True,padding=True,max_length=MAX_SEQUENCE_LENGTH) 
val_encod = bert_tokenizer(x_dev,truncation=True,padding=True,max_length=MAX_SEQUENCE_LENGTH)
test_encod = bert_tokenizer(x_test,truncation=True,padding=True,max_length=MAX_SEQUENCE_LENGTH)

class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

train_ds = NewsGroupsDataset(train_encod, y_train)
dev_ds = NewsGroupsDataset(val_encod, y_dev)
test_ds = NewsGroupsDataset(test_encod, y_test)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error


def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  #acc = accuracy_score(labels, preds)
  #f1 = f1_score(labels,preds)
  # mse = mean_squared_error(labels,pred.predictions,squared=False)
  return {
     #  'accuracy': acc,
     #  'f1_score':f1,
      'rmse': mse
  }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    metric_for_best_model = 'rmse',
    logging_steps=200,               # log & save weights each logging_steps
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
trainer = Trainer(
    model=bert_model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_ds,         # training dataset
    eval_dataset=dev_ds,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Rmse
200,0.353900,0.449304,0.670301
400,0.213200,0.479600,0.692531
600,0.361600,0.480087,0.692883
800,0.346600,0.429327,0.655230
1000,0.275200,0.472948,0.687712
1200,0.167700,0.457578,0.676445


TrainOutput(global_step=1389, training_loss=0.26800040841703365, metrics={'train_runtime': 625.055, 'train_samples_per_second': 2.222, 'total_flos': 200908559669292.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -488660992, 'train_mem_gpu_alloc_delta': 880645120, 'train_mem_cpu_peaked_delta': 488669184, 'train_mem_gpu_peaked_delta': 1349762048})

In [ ]:
trainer.evaluate(eval_dataset=test_ds)

{'epoch': 3.0,
 'eval_loss': 0.48496299982070923,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 71310848,
 'eval_rmse': 0.6963928937911987,
 'eval_runtime': 10.1092,
 'eval_samples_per_second': 231.67}

In [ ]:
trainer.save_model('best_model_BETO_rmse.hdf5')

Results:

    Step	Training Loss	Validation Loss	Accuracy	F1 Score
    200	0.513800	0.443248	0.782083	0.758764
    400	0.392800	0.337235	0.857083	0.803326
    600	0.376500	0.339753	0.847708	0.784047
    800	0.374800	0.345030	0.859167	0.801293
    1000	0.381600	0.327242	0.861042	0.824704
    1200	0.358000	0.347017	0.859375	0.822322
    1400	0.264400	0.393742	0.842292	0.813409
    1600	0.249500	0.360212	0.871875	0.830438
    1800	0.239200	0.387446	0.863958	0.828564
    2000	0.247400	0.336539	0.870417	0.832345
    2200	0.229900	0.384619	0.866250	0.831230
    2400	0.238700	0.345918	0.867083	0.831038



Results test:

    {'epoch': 2.0,
    'eval_accuracy': 0.8473333333333334,
    'eval_f1_score': 0.8133659331703341,
    'eval_loss': 0.3893824815750122,
    'eval_mem_cpu_alloc_delta': 0,
    'eval_mem_cpu_peaked_delta': 0,
    'eval_mem_gpu_alloc_delta': 0,
    'eval_mem_gpu_peaked_delta': 135538176,
    'eval_runtime': 42.7059,
    'eval_samples_per_second': 140.496}





Results train:

    Step	Training Loss	Validation Loss	Rmse
    200	0.353900	0.449304	0.670301
    400	0.213200	0.479600	0.692531
    600	0.361600	0.480087	0.692883
    800	0.346600	0.429327	0.655230
    1000	0.275200	0.472948	0.687712
    1200	0.167700	0.457578	0.676445

Results test:

    {'epoch': 3.0,
    'eval_loss': 0.48496299982070923,
    'eval_mem_cpu_alloc_delta': 0,
    'eval_mem_cpu_peaked_delta': 0,
    'eval_mem_gpu_alloc_delta': 0,
    'eval_mem_gpu_peaked_delta': 71310848,
    'eval_rmse': 0.6963928937911987,
    'eval_runtime': 10.1092,
    'eval_samples_per_second': 231.67}


https://www.thepythoncode.com/article/finetuning-bert-using-huggingface-transformers-python 